In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
root = 'C:/Users/tolik/information_technology/third_year/practice_project/CoreAnalysis-ML'
data = pd.read_csv('{}/data_for_study/data.csv'.format(root))
data.head(25)

,task_id,photo_id,photo_type,segment_num,segment_type,segment_value
0,c6a24494_7931_4ca9_a148_f1fd9785b607,1012473,УФ,0,Свечение,Насыщенное
1,c6a24494_7931_4ca9_a148_f1fd9785b607,1012473,УФ,1,Свечение,Отсутствует
2,cfb8e7b2_7c98_4bd4_897d_0e5e95801bc9,1016625,УФ,0,Свечение,Карбонатное
3,a9d8f546_9046_46ce_ba17_abda344a0980,1013361,УФ,0,Свечение,Карбонатное
4,eb09a8aa_6c77_41de_8489_f5a390c789e8,1016727,УФ,0,Свечение,Карбонатное
5,d9d6afc4_a213_4ae7_b86f_05d6c99457ef,1003985,УФ,0,Свечение,Карбонатное
6,d9d6afc4_a213_4ae7_b86f_05d6c99457ef,1003985,УФ,1,Свечение,Отсутствует
7,d9d6afc4_a213_4ae7_b86f_05d6c99457ef,1003985,УФ,2,Свечение,Карбонатное
8,e81388bd_c9f5_4164_9035_984ee3630ae5,1015963,УФ,0,Свечение,Отсутствует
9,e8f67b1c_6869_4148_9b90_f785374027dd,1014063,УФ,0,Свечение,Карбонатное


In [17]:
all_photos = data['photo_id'].unique().tolist()
all_tasks = data['task_id'].unique().tolist()
ultraviolet_photos=data[data['photo_type']=='УФ']['photo_id'].unique().tolist()
daylight_photos=data[data['photo_type']=='ДС']['photo_id'].unique().tolist()


print('Уникальных photo_id: {}\n'
      'Уникальных task_id: {}\n'
      'Фотографий в УФ: {}\n'
      'Фотографий в ДС: {}\n'
      .format(len(all_photos), len(all_tasks),
              len(ultraviolet_photos),len(daylight_photos)))

Уникальных photo_id: 2060
Уникальных task_id: 2060
Фотографий в УФ: 760
Фотографий в ДС: 1300



In [4]:
data_ultra = data[data['photo_type']=='УФ']
data_ultra['segment_num'].max()
## нужно заполнять матрицы значениями выше нижеприведенного

16

In [9]:
data.groupby(['segment_value']).count()

,task_id,photo_id,photo_type,segment_num,segment_type
segment_value,,,,,
Алевролит,9,9,9,9,9
Алевролит глинистый,301,301,301,301,301
Аргиллит,757,757,757,757,757
Аргиллит углистый,15,15,15,15,15
Глина аргиллитоподобная,4,4,4,4,4
Глинисто-кремнистая порода,16,16,16,16,16
Известняк,1,1,1,1,1
Карбонатная порода,5,5,5,5,5
Карбонатное,197,197,197,197,197


In [12]:
delete = []
one = data[data['segment_value'].isin(['Карбонатная порода', 'Известняк',
                                       'Глинисто-кремнистая порода',
                                       'Глина аргиллитоподобная',
                                      'Аргиллит углистый', 'Уголь', 'Алевролит'])]
delete = one['photo_id'].unique().tolist()      

In [16]:
len(delete)

31

In [23]:
photo_for_model = list(set(all_photos) - set(delete))

In [27]:
import cv2
h_s = []
w_s = []
root = 'C:/Users/tolik/information_technology/third_year/practice_project/CoreAnalysis-ML/data_for_study/photos/'
for photo_id in photo_for_model[:100]:
    image = cv2.imread(root + '{}.jpeg'.format(photo_id))
    print(image.shape)

(2737, 471, 3)
(3833, 602, 3)
(1770, 721, 3)
(6745, 725, 3)
(7204, 727, 3)
(7198, 733, 3)
(2169, 718, 3)
(5879, 785, 3)
(6034, 788, 3)
(6729, 795, 3)
(6693, 758, 3)
(2445, 715, 3)
(7408, 740, 3)
(833, 577, 3)
(7704, 731, 3)
(7023, 738, 3)
(7012, 723, 3)
(6042, 704, 3)
(1421, 573, 3)
(7359, 577, 3)
(7327, 581, 3)
(5077, 571, 3)
(7370, 677, 3)
(1058, 681, 3)
(7659, 710, 3)
(4015, 683, 3)
(1679, 602, 3)
(7251, 622, 3)
(5678, 622, 3)
(5376, 689, 3)
(5379, 680, 3)
(5739, 685, 3)
(2994, 694, 3)
(5004, 588, 3)
(2338, 564, 3)
(7515, 673, 3)
(7545, 692, 3)
(7412, 695, 3)
(7496, 682, 3)
(1810, 670, 3)
(7311, 549, 3)
(7280, 685, 3)
(6768, 585, 3)
(2143, 658, 3)
(4601, 562, 3)
(4102, 661, 3)
(3313, 688, 3)
(7688, 659, 3)
(7478, 657, 3)
(7374, 692, 3)
(7363, 674, 3)
(7325, 677, 3)
(3396, 618, 3)
(7348, 682, 3)
(7347, 674, 3)
(7847, 636, 3)
(7841, 652, 3)
(1775, 649, 3)
(7722, 662, 3)
(7624, 642, 3)
(7629, 653, 3)
(7651, 648, 3)
(7713, 654, 3)
(3545, 637, 3)
(6954, 633, 3)
(7559, 602, 3)
(7559, 632,

2060